In [1]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [2]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_All = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Merge Data\merged_data.csv')

print(df_All.head())


                  Date  425_pct_Al   M_Value  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-14 07:00:00     6.26763  3.551350    2.353670    21.66280   
1  2012-05-14 08:00:00     6.26798  3.551080    2.355025    21.66545   
2  2012-05-14 09:00:00     6.26868  3.550545    2.357740    21.67070   
3  2012-05-14 10:00:00     6.26938  3.550010    2.360455    21.67595   
4  2012-05-14 11:00:00     6.27008  3.549475    2.363170    21.68125   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O   Decanol      FC55003  \
0            0.519068    0.620812     1.088680  0.959336  6332.030000   
1            0.553096    0.621150     1.084100  0.959336  6332.030000   
2            0.621152    0.621824     1.074945  0.959336  6332.030000   
3            0.667952    0.622498     1.065790  0.933602  6191.103333   
4            0.693497    0.623173     1.056635  0.907868  6081.507500   

     FC55009       FC55552      FC55569  FFC55553  FFC55555    LC90366  \
0  2512.3700  41080.000000  6432.660000  1.000685  0.7

In [3]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [4]:
# # # List of columns to exclude from the XGboost feature selection results
# exclude_columns = [
#                     'TI40050',
#                     'TC55555',
#                     'LC55568',
#                     'LC55557',
#                     'Al2O3',
#                     'LC55553',
#                     'M_Value',
#                     'FFC55553',
#                     'DI55580',
#                     'Al2O3',
#                     'C4_pct_Hex',
#                     'LC52572',

# #                  'Date',
# #                  'C4_pct_Hex', 'HydWtr_Na2O',
# #                  'TC55555'  
#                    ]

# # # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [5]:
df_All.columns

Index(['Date', '425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'Decanol', 'FC55003',
       'FC55009', 'FC55552', 'FC55569', 'FFC55553', 'FFC55555', 'LC90366',
       'LC90368', 'PI55020', 'TC55552', 'TI55021'],
      dtype='object')

In [6]:
# df_All = df_All[df_All['Date'] > '2020-06-15 00:00:00']

In [7]:
# df_All.drop('Date', axis=1, inplace=True)




In [8]:
print(df_All.describe())

         425_pct_Al       M_Value    C4_pct_Eth    C4_pct_H2O  \
count  52974.000000  52974.000000  52974.000000  52974.000000   
mean       6.145939      3.590823      1.204019     21.209348   
std        0.254677      0.176865      0.689281      2.735901   
min        5.167535      1.347070      0.080626      7.932310   
25%        6.004946      3.500930      0.623279     18.876100   
50%        6.141118      3.586043      1.097525     21.481425   
75%        6.289489      3.678295      1.664852     23.292687   
max        7.131650      5.827565     11.732650     35.028700   

       HydWtr_pct_Ammonia    C4_pct_Hex   HydWtr_Na2O       Decanol  \
count        52974.000000  52974.000000  52974.000000  52974.000000   
mean             0.971848      0.460162      0.813779      3.179127   
std              0.152671      0.237337      0.709024      1.636934   
min              0.369965      0.000006      0.002854      0.087500   
25%              0.877023      0.340993      0.452738      

In [9]:
# # Assuming your data is in filtered_df and you want to predict 'target_column_name'
# df = df_All.drop(['Butanol', 'FC55009', 'PI55004', 'PI55020','TI55021'], axis=1)  # Replace 'target_column_name' with your target column's name
# # y = df_All['Decanol']

In [10]:
# # Keep only rows with specified IDs
# feature_names = [
# '425_pct_Al',
# 'C4_pct_Eth',
# 'C4_pct_H2O',
# 'HydWtr_pct_Ammonia',
# 'HydWtr_Na2O',
# 'DI55102',
# 'DI55152',
# 'FC55003',
# 'FC55009',
# 'FC55552',
# 'FC55569',
# 'FC55576',
# 'FFC55555',
# 'LC90366',
# 'LC90368',
# 'PI55004',
# 'PI55020',
# 'TC55552',
# 'TI55021'
# ]
    

In [11]:
import pickle

# Work
# # Load the trained XGBoost models
# with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Updated Model\C10_xgb_model.sav', 'rb') as model_file:
#     c10_model = pickle.load(model_file)
# with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C4\5 Preprocessing - Slecected Features\Updated Model\C4_xgb_model.sav', 'rb') as model_file:
#     c4_model = pickle.load(model_file)
# with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model STEAM - FC55569\5 Preprocessing - Slecected Features\Updated Model\STEAM_xgb_model.sav', 'rb') as model_file:
#     steam_model = pickle.load(model_file)

# Home
# Load the trained XGBoost models
with open(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Updated Model\C10_xgb_model.sav', 'rb') as model_file:
    c10_model = pickle.load(model_file)

with open(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\III Models\3 Model C4\5 Preprocessing - Slecected Features\Updated Model\C4_xgb_model.sav', 'rb') as model_file:               
    c4_model = pickle.load(model_file)

with open(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\III Models\3 Model STEAM - FC55569\5 Preprocessing - Slecected Features\Updated Model\STEAM_xgb_model.sav', 'rb') as model_file:
    steam_model = pickle.load(model_file)

# Initialize the new columns in the original df
df_All['FFC55555_adj'] = df_All['FFC55555']
df_All['Decanol_adj'] = None  # Using None as a placeholder
df_All['Butanol_adj'] = None  # Using None as a placeholder
df_All['FC55569_adj'] = None  # Using None as a placeholder

def predict_value(model, row, feature_names):
    # Ensure 'row' is prepared correctly for your model
    data_for_prediction = row[feature_names].values.reshape(1, -1)
    prediction = model.predict(data_for_prediction)[0]
    return prediction

# Assume 'decanol_feature_names' and 'butanol_feature_names' are defined
# You should define these lists based on the features your models expect

decanol_feature_names = ['425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'FC55003',
       'FC55009', 'FC55552', 'FC55569', 'FFC55553', 'FFC55555', 'LC90366',
       'LC90368', 'PI55020', 'TC55552', 'TI55021']  
butanol_feature_names = ['425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
       'C4_pct_Hex', 'HydWtr_Na2O', 'DI55102', 'DI55152', 'FC55552',
       'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC55557', 'LC90366',
       'LC90368', 'TC55552']
steam_feature_names = ['425_pct_Al', 'Al2O3', 'C4_pct_Eth', 'C4_pct_H2O', 'C4_pct_Hex',
       'DI55102', 'DI55152', 'DI55580', 'FC55003', 'FC55576',
       'FFC55555', 'LC90366', 'PI55020']

# decanol_feature_names = ['425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
#        'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'FC55003',
#        'FC55009', 'FC55552', 'FC55569', 'FFC55553', 'FFC55555', 'LC90366',
#        'LC90368', 'PI55020', 'TC55552', 'TI55021']  
# butanol_feature_names = ['425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
#        'C4_pct_Hex', 'HydWtr_Na2O', 'DI55102', 'DI55152', 'FC55552',
#        'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC55557', 'LC90366',
#        'LC90368', 'TC55552']
# steam_feature_names = ['425_pct_Al', 'Al2O3', 'C4_pct_Eth', 'C4_pct_H2O', 'C4_pct_Hex',
#        'DI55102', 'DI55152', 'DI55580', 'FC55003', 'FC55576',
#        'FFC55555', 'LC90366', 'PI55020']

# Your prediction and adjustment loop
for index, row in df_All.iterrows():
    # Adjust row for prediction if necessary
    row_for_prediction = row.copy()
    row_for_prediction['FFC55555'] = row_for_prediction['FFC55555_adj']
    
    # Process record through model C10 for Decanol
    df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
    
    # Process record through model C4 for Butanol
    df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)

    
# Initialize the starting value_rC4
value_rC4 = 0.68

# Specify the target_rC4 value_rC4
target_rC4 = 0.80

# Loop until value_rC4 reaches the target_rC4
while value_rC4 < target_rC4:
    print(f'Current value_rC4: {value_rC4:.2f}')
    value_rC4 += 0.01

    # Process record through Optimization Logic
    while True:
        decanol = df_All.at[index, 'Decanol_adj']
        butanol = df_All.at[index, 'Butanol_adj']
        ffc_adjusted = df_All.at[index, 'FFC55555_adj']

        if decanol < 0.01:
            if ffc_adjusted >= value_rC4: # Can be adjusted to the lowest rC4 desired (desired -0.01)
                df_All.at[index, 'FFC55555_adj'] -= 0.01
            else:
                break  # Next Record
        else:
            while decanol >= 0.01:
                if df_All.at[index, 'FFC55555'] >= ffc_adjusted:
                    break  # Next Record
                else:
                    if ffc_adjusted <= 1.19: # Can be adjusted to the highest rC4 desired
                        df_All.at[index, 'FFC55555_adj'] += 0.01
                    else:
                        break
            break

        # Update predictions after adjustment
        row_for_prediction['FFC55555'] = df_All.at[index, 'FFC55555_adj']
        df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
        

    # Update the FC55569_adj column with the output from the steam model
    df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)
    df_All.at[index, 'FC55569_adj'] = predict_value(steam_model, row_for_prediction, steam_feature_names)

KeyError: "['DI55102', 'DI55152', 'FC55576', 'LC55557'] not in index"

In [ ]:
df_All.head()

In [ ]:

df_steam = df_All[['Date', 'FC55569', 'FC55569_adj']]

df_steam['df_steam_diff'] = df_steam['FC55569'] - df_steam['FC55569_adj']

df_steam

In [ ]:
# Sum each column
column_sums = df_steam.sum()

# Convert the sums into a DataFrame
sum_row = pd.DataFrame(column_sums).transpose()

# Add a label to the sum row
sum_row['Date'] = 'Sum'

# Append the sum row to the original DataFrame
df_steam_with_sum = pd.concat([df_steam, sum_row], ignore_index=True)

# Display the result
df_steam_with_sum


That might be more steam, but How much alcohol is saved?????????

In [ ]:
df_C10 = df_All[['Date', 'Decanol', 'Decanol_adj']]

df_C10['Decanol_diff'] = df_C10['Decanol'] - df_C10['Decanol_adj']

df_C10

In [ ]:
# Sum each column
column_sums = df_C10.sum()

# Convert the sums into a DataFrame
sum_row = pd.DataFrame(column_sums).transpose()

# Add a label to the sum row
sum_row['Date'] = 'Sum'

# Append the sum row to the original DataFrame
df_C10_with_sum = pd.concat([df_C10, sum_row], ignore_index=True)

# Display the result
df_C10_with_sum


In [ ]:
df_C4 = df_All[['Date', 'Butanol', 'Butanol_adj']]

df_C4['Butanol_diff'] = df_C4['Butanol'] - df_C4['Butanol_adj']

df_C4

In [ ]:
# Sum each column
column_sums = df_C4.sum()

# Convert the sums into a DataFrame
sum_row = pd.DataFrame(column_sums).transpose()

# Add a label to the sum row
sum_row['Date'] = 'Sum'

# Append the sum row to the original DataFrame
df_C4_with_sum = pd.concat([df_C4, sum_row], ignore_index=True)

# Display the result
df_C4_with_sum
